In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import f
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
import statistics
from sklearn.preprocessing import MinMaxScaler

import joblib

In [4]:
df = pd.read_csv('/Users/tehuttesfayebiru/Desktop/MCGILL/Program/Winter term/W2/INSY 675 - EDS/clustering_df.csv').iloc[:,1:]
df

,store_nbr,BABY CARE,BEVERAGES,BOOKS,BREAD/BAKERY,CLEANING,DAIRY,DELI,FROZEN FOODS,GROCERIES,...,LADIESWEAR,LAWN AND GARDEN,"LIQUOR,WINE,BEER",MEATS,OTHER,PERSONAL CARE,POULTRY,PRODUCE,transactions,total_sales
0,1,0.000000,18.902748,0.001492,4.029323,7.624483,7.453233,1.463511,1.540569,26.730029,...,0.112006,0.107227,1.202565,3.729644,3.706572,1.537917,3.383659,17.271545,2553963,1.414660e+07
1,2,0.000389,19.260932,0.001108,4.285650,7.539144,6.662193,1.907932,1.254980,30.841461,...,0.137011,0.064282,0.675594,3.940995,3.283531,2.059788,3.799749,12.898353,3219901,2.156755e+07
2,3,0.001331,22.487688,0.001070,3.837526,6.882394,6.807689,1.316565,1.498906,25.952200,...,0.103020,0.081837,0.732337,2.706705,3.994024,1.679733,3.773844,17.061038,5366350,5.048553e+07
3,4,0.000127,19.400316,0.001407,3.611438,7.878741,7.028889,1.923636,1.341576,29.991410,...,0.135344,0.083385,0.740429,3.291591,3.216057,2.030527,3.997834,13.958046,2519007,1.891105e+07
4,5,0.001379,16.251393,0.001475,4.122759,10.695893,5.587790,2.663210,1.729505,33.964111,...,0.095037,0.031874,0.708336,3.059760,3.369688,2.887050,2.691282,10.604885,2347877,1.559287e+07
5,6,0.000048,20.105626,0.000302,3.649490,8.263724,6.400918,2.284553,1.503467,31.234584,...,0.128879,0.052059,0.624399,3.312038,3.230995,2.148934,4.219885,11.488752,3065896,2.518486e+07
6,7,0.000178,20.756317,0.000783,4.014808,6.569456,7.654521,1.055825,1.609857,23.432219,...,0.045485,0.118902,0.995493,3.264670,3.741443,1.332629,3.778905,20.688814,2995993,2.695588e+07
7,8,0.000466,20.813785,0.001039,4.156867,6.965445,7.784193,1.538043,1.311913,26.167399,...,0.137061,0.092856,0.815148,2.867690,4.050347,1.810739,3.305751,16.894475,4637971,3.049667e+07
8,9,0.001272,17.483456,0.000000,3.619308,10.977639,4.755176,3.727508,0.795246,39.278832,...,0.060431,0.026840,0.327716,2.403355,2.610388,2.826895,3.085204,6.179209,3516162,2.641187e+07
9,10,0.004795,15.680140,0.000000,1.991028,13.624083,4.395185,4.364501,0.761980,42.983643,...,0.057103,0.000270,0.213133,3.272983,1.750321,3.128597,3.067885,2.608012,1652493,9.614181e+06


In [8]:
df.columns

Index(['store_nbr', 'BABY CARE', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY',
       'CLEANING', 'DAIRY', 'DELI', 'FROZEN FOODS', 'GROCERIES', 'HOME CARE',
       'LADIESWEAR', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEATS', 'OTHER',
       'PERSONAL CARE', 'POULTRY', 'PRODUCE', 'transactions', 'total_sales'],
      dtype='object')

In [14]:

print('Starting clustering')
# Importing the clustering dataset
# Dropping store_nbr
clustering_df = df.drop(columns= 'store_nbr')
# Standardizing the transactions and total_sales columns
scaler = MinMaxScaler()
scaler.fit(clustering_df[['total_sales']])
clustering_df['total_sales'] = scaler.transform(clustering_df[['total_sales']])

# Choosing the variables
X = clustering_df[['CLEANING', 'DAIRY', 'GROCERIES', 'PRODUCE', 'POULTRY',
'FROZEN FOODS','DELI','BREAD/BAKERY','OTHER', 'total_sales']]

X_cols = list(X.columns)

# Creating a loop to vary 'k' in order to plot the elbow and silhouette plot and select the optimal no. of clusters
list_base_model = []
withinss=[]
#     for i in range(2,20):
#         kmeans = KMeans(n_clusters=i, random_state=0)
#         model = kmeans.fit(X)
#         labels = model.labels_
#         withinss.append(model.inertia_)
#         silhouette = silhouette_score(X, labels)
#         list_base_model.append(silhouette)

#pyplot.plot(list(range(2,20)), withinss, marker= 'o')

#pyplot.plot(list(range(2,20)), list_base_model, marker= 'o')


# Running the final K means model with k=4
kmeans = KMeans(n_clusters=4, random_state=0)
model = kmeans.fit(X)
labels = model.labels_


silhouette_sam = silhouette_samples(X, labels)

#     df_100 = pd.DataFrame({'label':labels,'silhouette':silhouette_sam}) # save silhouette score
#     print('Average Silhouette Score for Cluster 0: ',np.average(df_100[df_100['label'] == 0].silhouette))
#     print('Average Silhouette Score for Cluster 1: ',np.average(df_100[df_100['label'] == 1].silhouette))
#     print('Average Silhouette Score for Cluster 2: ',np.average(df_100[df_100['label'] == 2].silhouette))
#     print('Average Silhouette Score for Cluster 3: ',np.average(df_100[df_100['label'] == 3].silhouette))


#     # Creating a dataframe to store the cluster centers
#     cluster_centers_df = pd.DataFrame(model.cluster_centers_)

#     cluster_centers_df.columns = X_cols

#     # Adding the clusters column to our original dataframe
#     df['assigned_cluster'] = labels

#     df.to_csv('docker_data/outputs/store_labels.csv', index=None)
#     cluster_centers_df.to_csv('docker_data/outputs/cluster_summary.csv', index=None)
#     df_100.to_csv('docker_data/outputs/cluster_sil_scores.csv', index = None)


#     joblib.dump(kmeans, 'docker_data/outputs/kmeans.pkl')
#     joblib.dump(scaler, 'docker_data/outputs/scaler.pkl')

# return None


# # if __name__ == '__main__':
# get_clusters()

Starting clustering


In [11]:
pip install deepchecks

     |████████████████████████████████| 356 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 26.5 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.2 MB/s eta 0:00:011
     |████████████████████████████████| 7.9 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 123 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 9.5 MB/s eta 0:00:01     |███████                         | 348 kB 9.5 MB/s eta 0:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.1
    Uninstalling widgetsnbextension-3.5.1:
      Successfully uninstalled widgetsnbextension-3.5.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
   

In [13]:
pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install scikit-learn 

  Using cached scikit_learn-1.0.2-cp38-cp38-macosx_10_13_x86_64.whl (7.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from deepchecks import Dataset
from deepchecks.suites import full_suite

/Users/tehuttesfayebiru/opt/anaconda3/lib/python3.8/site-packages/deepchecks/__init__.py:141: DeprecationWarning:

Ability to import base tabular functionality from the `deepchecks` package directly is deprecated, please import from `deepchecks.tabular` instead

/Users/tehuttesfayebiru/opt/anaconda3/lib/python3.8/site-packages/deepchecks/suites.py:21: DeprecationWarning:

Ability to import tabular suites from the `deepchecks.suites` is deprecated, please import from `deepchecks.tabular.suites` instead



In [18]:
# ds_train = X_train.merge(y_train, left_index=True, right_index=True)
# ds_test = X_test.merge(y_test, left_index=True, right_index=True)
ds_train = Dataset(X)
# ds_test =  Dataset(ds_test,  label="WasTheLoanApproved", cat_features=["TypeOfCurrentEmployment", "LoanReason", "DebtsPaid"])
suite = full_suite()
result = suite.run(train_dataset=ds_train, model=model)
result

Full Suite:   0%|          | 0/35 [00:00<?, ? Check/s]

/Users/tehuttesfayebiru/opt/anaconda3/lib/python3.8/site-packages/deepchecks/utils/features.py:106: UserWarning:

Features importance was not calculated:
Expected dataset with label.



In [19]:
result_json = result.to_json()
result_json

'{"name": "Full Suite", "results": ["{\\"name\\": \\"Model Info\\", \\"params\\": {}, \\"header\\": \\"Model Info\\", \\"summary\\": \\"Summarize given model parameters.\\", \\"value\\": {\\"type\\": \\"KMeans\\", \\"params\\": {\\"algorithm\\": \\"auto\\", \\"copy_x\\": true, \\"init\\": \\"k-means++\\", \\"max_iter\\": 300, \\"n_clusters\\": 4, \\"n_init\\": 10, \\"random_state\\": 0, \\"tol\\": 0.0001, \\"verbose\\": 0}}, \\"display\\": [[\\"html\\", \\"Model Type: KMeans\\"], [\\"dataframe\\", \\"[{\\\\\\"Parameter\\\\\\":\\\\\\"algorithm\\\\\\",\\\\\\"Value\\\\\\":\\\\\\"auto\\\\\\",\\\\\\"Default\\\\\\":\\\\\\"auto\\\\\\"},{\\\\\\"Parameter\\\\\\":\\\\\\"copy_x\\\\\\",\\\\\\"Value\\\\\\":true,\\\\\\"Default\\\\\\":true},{\\\\\\"Parameter\\\\\\":\\\\\\"init\\\\\\",\\\\\\"Value\\\\\\":\\\\\\"k-means++\\\\\\",\\\\\\"Default\\\\\\":\\\\\\"k-means++\\\\\\"},{\\\\\\"Parameter\\\\\\":\\\\\\"max_iter\\\\\\",\\\\\\"Value\\\\\\":300,\\\\\\"Default\\\\\\":300},{\\\\\\"Parameter\\\\\\":\\\\\